In [0]:
import pandas as pd
import numpy as np
import nltk
from sklearn.feature_extraction.text import ENGLISH_STOP_WORDS
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
import re
import random 
import collections
from gensim.models import KeyedVectors
from time import time
from gensim.models import KeyedVectors
import re
from nltk.corpus import stopwords
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import seaborn as sns
import itertools
import datetime
from keras.models import Model
from keras.layers import Input, Embedding, LSTM, Lambda, CuDNNLSTM
import keras.backend as K
from keras.optimizers import Adadelta
from keras.preprocessing.sequence import pad_sequences
from keras.callbacks import TensorBoard
embedding_dim = 300
n_hidden = 50
gradient_clipping_norm = 1.25
batch_size = 64
n_epoch = 10

In [0]:
from google.colab import drive
drive.mount('/gdrive')

In [0]:
data1 = pd.read_csv("/gdrive/My Drive/train.csv", delimiter=',', encoding="utf-8-sig")
data2 = pd.read_csv("/gdrive/My Drive/test.csv", delimiter=',', encoding="utf-8-sig")
EMBEDDING_FILE = '/gdrive/My Drive/GoogleNews-vectors-negative300.bin.gz'
word2vec = KeyedVectors.load_word2vec_format(EMBEDDING_FILE, binary=True)
print(data1.shape)
print(data1.iloc[0:1,3:])
#random.shuffle(data1)

In [0]:
subset = data1.iloc[0:100,3:]
subset
#subset = subset.loc[subset.is_duplicate==0,['question1','question2']]

In [0]:
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords # Import the stop word list
print(stopwords.words("english"))

In [0]:
def question_to_words( raw_quest ):
    lemmetizer = nltk.WordNetLemmatizer()
    letters_only = re.sub("[^a-zA-Z]", " ", raw_quest) 
    letters_only = letters_only.lower()
    words = nltk.word_tokenize(letters_only)
    #words = nltk.word_tokenize(s)
    stops = set(stopwords.words("english"))                  
    meaningful_words = [w for w in words if not w in stops]
    meaningful_words = [lemmetizer.lemmatize(w) for w in meaningful_words]
    return( " ".join( meaningful_words ))

In [0]:
nltk.download('punkt')
nltk.download('wordnet')
subset['question1']= subset['question1'].apply(question_to_words)
subset['question2']= subset['question2'].apply(question_to_words)

In [0]:
X = subset.iloc[:,0:-1]
X
y = subset.iloc[:,-1]



In [0]:
def text_to_word_list(text): #Standard code used for cleaning data.
    ''' Pre process and convert texts to a list of words '''
    text = str(text)
    text = text.lower()

    # Clean the text
    text = re.sub(r"[^A-Za-z0-9^,!.\/'+-=]", " ", text)
    text = re.sub(r"what's", "what is ", text)
    text = re.sub(r"\'s", " ", text)
    text = re.sub(r"\'ve", " have ", text)
    text = re.sub(r"can't", "cannot ", text)
    text = re.sub(r"n't", " not ", text)
    text = re.sub(r"i'm", "i am ", text)
    text = re.sub(r"\'re", " are ", text)
    text = re.sub(r"\'d", " would ", text)
    text = re.sub(r"\'ll", " will ", text)
    text = re.sub(r",", " ", text)
    text = re.sub(r"\.", " ", text)
    text = re.sub(r"!", " ! ", text)
    text = re.sub(r"\/", " ", text)
    text = re.sub(r"\^", " ^ ", text)
    text = re.sub(r"\+", " + ", text)
    text = re.sub(r"\-", " - ", text)
    text = re.sub(r"\=", " = ", text)
    text = re.sub(r"'", " ", text)
    text = re.sub(r"(\d+)(k)", r"\g<1>000", text)
    text = re.sub(r":", " : ", text)
    text = re.sub(r" e g ", " eg ", text)
    text = re.sub(r" b g ", " bg ", text)
    text = re.sub(r" u s ", " american ", text)
    text = re.sub(r"\0s", "0", text)
    text = re.sub(r" 9 11 ", "911", text)
    text = re.sub(r"e - mail", "email", text)
    text = re.sub(r"j k", "jk", text)
    text = re.sub(r"\s{2,}", " ", text)

    text = text.split()

    return text
stops = set(stopwords.words('english'))

In [0]:

vocabulary = dict()
vocabulary_T = ['<unk>']  
questions_cols = ['question1', 'question2']

# Run both traing and test data sets.
for dataset in [data1, data2]:
    for index, row in dataset.iterrows():

        #Use both the questions in 'x' row
        for question in questions_cols:

            qc = []  
            for word in text_to_word_list(row[question]):

                if word in stops and word not in word2vec.vocab:
                    continue

                if word not in vocabulary:
                    vocabulary[word] = len(vocabulary_T)
                    qc.append(len(vocabulary_T))
                    vocabulary_T.append(word)
                else:
                    qc.append(vocabulary[word])

            dataset.set_value(index, question, qc)
            

embeddings = 1 * np.random.randn(len(vocabulary) + 1, embedding_dim)  # This will be the embedding matrix
embeddings[0] = 0  # To ignore padding

# Matrix
for word, index in vocabulary.items():
    if word in word2vec.vocab:
        embeddings[index] = word2vec.word_vec(word)

In [0]:
X = data1[questions_cols]

Y = data1['is_duplicate']

X_train, X_test, y_train, y_test = train_test_split(X, Y,test_size=0.2) # validation split

In [0]:
max_seq_length = max(X_train.question1.map(lambda x: len(x)).max(),
                     X_train.question2.map(lambda x: len(x)).max(),
                     X_test.question2.map(lambda x: len(x)).max(),
                    X_test.question2.map(lambda x: len(x)).max())

In [0]:
X_train = {'left': X_train.question1, 'right': X_train.question2}
X_test = {'left': X_test.question1, 'right': X_test.question2}
y_train = y_train.values
y_test = y_test.values
for dataset, side in itertools.product([X_train, X_test], ['left', 'right']):
    dataset[side] = pad_sequences(dataset[side], maxlen=max_seq_length)

In [0]:

n_hidden = 50
gradient_clipping_norm = 1.25
batch_size = 64
n_epoch = 10

def exponent_neg_manhattan_distance(left, right):
    return K.exp(-K.sum(K.abs(left-right), axis=1, keepdims=True))

left_input = Input(shape=(max_seq_length,), dtype='int32')
right_input = Input(shape=(max_seq_length,), dtype='int32')

In [0]:
#For TensorBoard
import time
log_dir="/gdrive/My Drive/logs/{}".format(time.strftime("%Y%m%d-%H%M%S", time.gmtime()))
tensorboard = TensorBoard(log_dir=log_dir, histogram_freq=1)

In [0]:
# To get f1 score
from keras import backend as K

def f1(y_true, y_pred):
    def recall(y_true, y_pred):
        true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
        possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
        recall = true_positives / (possible_positives + K.epsilon())
        return recall

    def precision(y_true, y_pred):
        true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
        predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
        precision = true_positives / (predicted_positives + K.epsilon())
        return precision
    precision = precision(y_true, y_pred)
    recall = recall(y_true, y_pred)
    return 2*((precision*recall)/(precision+recall+K.epsilon()))


In [0]:
embedding_layer = Embedding(len(embeddings), embedding_dim, weights=[embeddings], input_length=max_seq_length, trainable=False)

# left right inputs feeded to embedding
encoded_left = embedding_layer(left_input)
encoded_right = embedding_layer(right_input)

# *****Using GPU*******
shared_lstm = CuDNNLSTM(n_hidden)

left_output = shared_lstm(encoded_left)
right_output = shared_lstm(encoded_right)

# Lambda Function for manhattan distance
malstm_distance = Lambda(function=lambda x: exponent_neg_manhattan_distance(x[0], x[1]),output_shape=lambda x: (x[0][0], 1))([left_output, right_output])

# Main Model Ready
malstm = Model([left_input, right_input], [malstm_distance])

# Adadelta optimizer
optimizer = Adadelta(clipnorm=gradient_clipping_norm)

malstm.compile(loss='mean_squared_error', optimizer=optimizer, metrics=['accuracy',f1])


malstm_trained = malstm.fit([X_train['left'], X_train['right']], y_train, batch_size=batch_size, nb_epoch=1,validation_data=([X_test['left'], X_test['right']], y_test), callbacks = [tensorboard])
print(malstm.summary())

In [0]:
from keras.utils import plot_model
plot_model(malstm, show_shapes=True, show_layer_names=True, to_file='model.png')
from IPython.display import Image
Image(retina=True, filename='model.png')

In [0]:
import matplotlib.pyplot as plt
fig1=plt.figure()
plt.plot(history.history['acc'],'r',linewidth=3.0)
plt.plot(history.history['val_acc'],'b',linewidth=3.0)
plt.legend(['Training Accuracy', 'Validation Accuracy'],fontsize=18)
plt.xlabel('Epochs ',fontsize=16)
plt.ylabel('Accuracy',fontsize=16)
plt.title('Accuracy Curves',fontsize=16)
fig1.savefig('accuracy.png')
plt.show()


In [0]:
fig2 = plt.figure()
plt.plot(history.history['loss'],'r',linewidth=3.0)
plt.plot(history.history['val_loss'],'b',linewidth=3.0)
plt.legend(['Training loss', 'Validation Loss'],fontsize=18)
plt.xlabel('Epochs ',fontsize=16)
plt.ylabel('Loss',fontsize=16)
plt.title('Loss Curves',fontsize=16)
fig2.savefig('loss.png')
plt.show()

In [0]:
fig3 = plt.figure()
plt.plot(history.history['f1'],'r',linewidth=3.0)
plt.plot(xhistory.history['val_f1'],'b',linewidth=3.0)
plt.legend(['F1 Training', 'F1 Validation'],fontsize=18)
plt.xlabel('Epochs ',fontsize=16)
plt.ylabel('F1',fontsize=16)
plt.title('F1 Curves',fontsize=16)
fig3.savefig('f1.png')
plt.show()